In [73]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

In [74]:
randomState = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=randomState, stratify=Y)

Using device: cuda


In [75]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, randomState=None, oversample=False):
    
    """
    Generates stratified folds with specified normalization.
    
    For list of scalers, see:
    https://scikit-learn.org/stable/api/sklearn.preprocessing.html
    
    For more details on scaling and normalization effects, see:
    https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#
    
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - Quantile_Transform(output_distribution='uniform')
    
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=randomState)
    kFolds_list = []
    
    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # Fit the scaler on the training data and transform both train and test sets
        X_train_scaled = normalisation_method.fit_transform(X_train)
        X_test_scaled = normalisation_method.transform(X_test)
        
        if oversample:
            # Oversample the training data if needed (e.g., using SMOTE or similar techniques)
            # This is a placeholder; actual oversampling code should be implemented here
            # X_train_scaled....
            pass
        
        # Convert back to DataFrame to maintain column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
        
        # Ensure 'gender' is still binary (0 or 1)
        if X_train_scaled['Gender'].isin([0, 1]).all():
            kFolds_list.append((X_train_scaled, X_test_scaled, Y_train, Y_test))
        else:
            print("Warning: 'gender' column contains unexpected values after scaling.") 
               
        print(f"Fold: {fold+1}, Train: {kFolds_list[fold][0].shape}, Test: {kFolds_list[fold][1].shape}")   
    return kFolds_list

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 

In [76]:
kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, normalisation_method=MinMaxScaler(), n_splits=5, randomState=randomState)

Fold: 1, Train: (4593, 28), Test: (1149, 28)
Fold: 2, Train: (4593, 28), Test: (1149, 28)
Fold: 3, Train: (4594, 28), Test: (1148, 28)
Fold: 4, Train: (4594, 28), Test: (1148, 28)
Fold: 5, Train: (4594, 28), Test: (1148, 28)


In [77]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from modularModels1 import BlockMaker, modularNN, BasicModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 


def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

Using cuda


In [78]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss()
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

In [79]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            # nn.Sigmoid()
   
        )

    def forward(self, x):
        return self.net(x)
    
    def last_layer(self):
        return self.net[-1]

In [80]:
# test_model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=64, dropout=0.5).to(device)
# print(get_feature_count(train_loader))

In [81]:
def train_and_evaluate(model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=20, patience=5, device=device):
    if isinstance(model.last_layer(), nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")

    
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    best_val_loss = float('inf')
    best_model_state = None
    wait = 0

    
    #* Epoch Training loop for this fold
    for epoch in range(1,epochs+1):
        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        running_loss = 0.0 #? loss for this epoch
        #* Mini-batch training loop
        for batch, (inputs, labels) in enumerate(train_loader,start=1):
            optimiser.zero_grad() #? Zero the gradients
            outputs = model(inputs) #? Forward pass through the model
            loss = criterion(outputs, labels) #? Calculate loss
            loss.backward() #? Backpropagation
            running_loss += loss.item()
            optimiser.step() #? Update weights
            if scheduler:
                scheduler.step()
                
        train_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
    
        #* Now we evaluate the model on the validation set, to track training vs validation loss
        model.eval() #? Set model to evaluation mode
        with torch.no_grad(): #? No need to track gradients during evaluation
            val_loss = 0.0    
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                # labels = labels.cpu() 
                loss = criterion(outputs, labels)
                val_loss += loss.item() #? Calculate loss
                avg_val_loss = val_loss / len(val_loader)
                print(f"Epoch {epoch}, Val Loss: {avg_val_loss:.4f}")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            wait = 0
        else:
            if avg_val_loss*0.95 <= best_val_loss:
                wait += 1
            if wait >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break
    
    #* Use best model to calculate metrics on the validation set
    #! must be outside epoch loop, it comes after the training and cv loop
    model.load_state_dict(best_model_state) #? Load the best model state
    with torch.no_grad():
        for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                labels = labels.cpu() 
                predictions = (torch.sigmoid(outputs) < 0.5).float().cpu().numpy()
                
                val_loss += loss.item() #? Calculate loss
                
    #! The following should have length equal to fold number           
    accuracy_list.append(accuracy_score(labels, predictions)) 
    precision_list.append(precision_score(labels, predictions, pos_label=1, zero_division=0)) 
    recall_list.append(recall_score(labels, predictions, pos_label=1))
    f1_list.append(f1_score(labels, predictions, pos_label=1))
    auc_list.append(roc_auc_score(labels, predictions)) 

    return model, accuracy_list, precision_list, recall_list, f1_list, auc_list 


In [82]:
def maximise_combined_score(trial):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    # Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 28, 128)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    initial_lr = trial.suggest_float("initial_lr", 1e-5, 1e-3, log=True)
    max_lr = trial.suggest_float("max_lr", 1e-3, 1e-1, log=True)
    
    # Loss function hyperparameters
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss"]) #, "FocalLoss", "DiceLoss"])
    
    # Hyperparameter exploration optimization
    if criterion_choice == "BCEWithLogitsLoss":
        pos_weight = trial.suggest_int("pos_weight", 1, 10)
    else:
        pos_weight = None
    
    # Initialize lists for metrics across folds
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []

    # Cross-validation loop
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        # Create DataLoader for current fold
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device)
        # Calculate steps_per_epoch from the current fold's train_loader
        train_loader_len = len(train_loader)
        
        # Instantiate and initialize the model
        model = BinaryClassifier(input_dim=get_feature_count(train_loader), hidden_dim=hidden_dim, dropout=dropout)
        model.to(device)
        model.apply(init_weights)
        
        # Map the choice to the actual loss function
        criterion = criterion_mapping(criterion_choice, pos_weight).to(device)
        optimiser = optim.Adam(model.parameters(), lr=initial_lr)
        
        # Initialize scheduler
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimiser,
            max_lr=max_lr,
            steps_per_epoch=train_loader_len,
            epochs=100,
            anneal_strategy='linear'
        )
        print(f"Fold {fold}:")
        # Train and evaluate the model on the current fold
        model, accuracy, precision, recall, f1, auc = train_and_evaluate(
            model, criterion, optimiser, scheduler, train_loader, val_loader, epochs=10, patience=10, device=device
        )

        # Append the metrics from the current fold
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
        auc_list.append(auc)

    # Calculate the average metrics across all folds
    avg_accuracy = np.sum(accuracy_list) / len(accuracy_list)
    avg_precision = np.sum(precision_list) / len(precision_list)
    avg_recall = np.sum(recall_list) / len(recall_list)
    avg_f1 = np.sum(f1_list) / len(f1_list)
    avg_auc = np.sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score"
    combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5

    return combined_score


In [84]:
import threading
import optuna
from optuna_dashboard import run_server

def start_dashboard():
    run_server(storage)

storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(direction="maximize", storage=storage, study_name="Basic")

# Start dashboard in a separate thread
dashboard_thread = threading.Thread(target=start_dashboard, daemon=True)
dashboard_thread.start()

# Run optimization
study.optimize(maximise_combined_score, n_trials=30)

# After optimization, print results
print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-07 19:17:24,154] A new study created in memory with name: Basic
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.



Using device: cuda
Fold 1:
Epoch: 1, training loss: 1.3699
Epoch 1, Val Loss: 1.3111
Epoch: 2, training loss: 1.2935
Epoch 2, Val Loss: 1.2683
Epoch: 3, training loss: 1.2696
Epoch 3, Val Loss: 1.2338
Epoch: 4, training loss: 1.2409
Epoch 4, Val Loss: 1.1977
Epoch: 5, training loss: 1.2212
Epoch 5, Val Loss: 1.1694
Epoch: 6, training loss: 1.1983
Epoch 6, Val Loss: 1.1422
Epoch: 7, training loss: 1.1789
Epoch 7, Val Loss: 1.1223
Epoch: 8, training loss: 1.1598
Epoch 8, Val Loss: 1.1172
Epoch: 9, training loss: 1.1425
Epoch 9, Val Loss: 1.1056
Epoch: 10, training loss: 1.1482
Epoch 10, Val Loss: 1.1254
Fold 2:
Epoch: 1, training loss: 1.3091
Epoch 1, Val Loss: 1.2916
Epoch: 2, training loss: 1.2821
Epoch 2, Val Loss: 1.2648
Epoch: 3, training loss: 1.2567
Epoch 3, Val Loss: 1.2336
Epoch: 4, training loss: 1.2230
Epoch 4, Val Loss: 1.2053
Epoch: 5, training loss: 1.1972
Epoch 5, Val Loss: 1.1823
Epoch: 6, training loss: 1.1755
Epoch 6, Val Loss: 1.1640
Epoch: 7, training loss: 1.1464
Epo

127.0.0.1 - - [07/Apr/2025 19:17:27] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [07/Apr/2025 19:17:27] "GET /dashboard HTTP/1.1" 200 4145


Epoch: 9, training loss: 1.1267
Epoch 9, Val Loss: 1.1643
Epoch: 10, training loss: 1.1252
Epoch 10, Val Loss: 1.1587
Fold 4:
Epoch: 1, training loss: 1.3338
Epoch 1, Val Loss: 1.2953
Epoch: 2, training loss: 1.3009
Epoch 2, Val Loss: 1.2696


127.0.0.1 - - [07/Apr/2025 19:17:27] "GET /static/bundle.js HTTP/1.1" 200 4140872


...

[I 2025-04-07 19:20:27,740] Trial 28 finished with value: 0.3677908916429346 and parameters: {'hidden_dim': 73, 'dropout': 0.35024667740703774, 'initial_lr': 0.00019647227200486015, 'max_lr': 0.003804973918051341, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 2}. Best is trial 13 with value: 0.37735558639428074.


Epoch: 10, training loss: 0.4727
Epoch 10, Val Loss: 0.4774
Using device: cuda
Fold 1:
Epoch: 1, training loss: 0.7661
Epoch 1, Val Loss: 0.7047
Epoch: 2, training loss: 0.6888
Epoch 2, Val Loss: 0.6687
Epoch: 3, training loss: 0.6710
Epoch 3, Val Loss: 0.6534
Epoch: 4, training loss: 0.6504
Epoch 4, Val Loss: 0.6369
Epoch: 5, training loss: 0.6494
Epoch 5, Val Loss: 0.6220
Epoch: 6, training loss: 0.6264
Epoch 6, Val Loss: 0.6083
Epoch: 7, training loss: 0.6155
Epoch 7, Val Loss: 0.5953
Epoch: 8, training loss: 0.6118
Epoch 8, Val Loss: 0.5892
Epoch: 9, training loss: 0.5990
Epoch 9, Val Loss: 0.5858
Epoch: 10, training loss: 0.5940
Epoch 10, Val Loss: 0.5831
Fold 2:
Epoch: 1, training loss: 0.7530
Epoch 1, Val Loss: 0.7106
Epoch: 2, training loss: 0.6947
Epoch 2, Val Loss: 0.6749
Epoch: 3, training loss: 0.6674
Epoch 3, Val Loss: 0.6550
Epoch: 4, training loss: 0.6497
Epoch 4, Val Loss: 0.6359
Epoch: 5, training loss: 0.6350
Epoch 5, Val Loss: 0.6224
Epoch: 6, training loss: 0.6172
E

[I 2025-04-07 19:20:33,583] Trial 29 finished with value: 0.3298145897323156 and parameters: {'hidden_dim': 43, 'dropout': 0.2916714612161655, 'initial_lr': 6.0470067299959614e-05, 'max_lr': 0.006214577365959155, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 3}. Best is trial 13 with value: 0.37735558639428074.


Epoch: 9, training loss: 0.6082
Epoch 9, Val Loss: 0.6158
Epoch: 10, training loss: 0.5984
Epoch 10, Val Loss: 0.6120
Best trial:
  Combined score: 0.37735558639428074
  Best hyperparameters:
    hidden_dim: 53
    dropout: 0.22349186907065982
    initial_lr: 6.995890036673634e-05
    max_lr: 0.08938960521453812
    criterion: BCEWithLogitsLoss
    pos_weight: 1


127.0.0.1 - - [07/Apr/2025 19:20:36] "GET /api/studies/0?after=28 HTTP/1.1" 200 7336
d:\GitHub repos\ADL\.venv\Lib\site-packages\optuna_dashboard\_importance.py:96: ExperimentalWarning: PedAnovaImportanceEvaluator is experimental (supported from v3.6.0). The interface can change in the future.
  study, target=target, evaluator=PedAnovaImportanceEvaluator()
d:\GitHub repos\ADL\.venv\Lib\site-packages\optuna\importance\_ped_anova\evaluator.py:150: UserWarning: PedAnovaImportanceEvaluator computes the importances of params to achieve low `target` values. If this is not what you want, please modify target, e.g., by multiplying the output by -1.
  warnings.warn(
127.0.0.1 - - [07/Apr/2025 19:20:37] "GET /api/studies/0/param_importances?evaluator=ped_anova HTTP/1.1" 200 580
127.0.0.1 - - [07/Apr/2025 19:20:47] "GET /api/studies/0?after=30 HTTP/1.1" 200 3690
127.0.0.1 - - [07/Apr/2025 19:20:58] "GET /api/studies/0?after=30 HTTP/1.1" 200 3690
127.0.0.1 - - [07/Apr/2025 19:21:09] "GET /api/stud

In [ ]:
import optuna
from optuna_dashboard import run_server

storage = optuna.storages.InMemoryStorage()

study = optuna.create_study(direction="maximize",storage=storage,  # Specify the storage URL here.
    study_name="Basic")
study.optimize(maximise_combined_score, n_trials=30)  # You can adjust the number of trials
run_server(storage)





print("Best trial:")
trial = study.best_trial
print(f"  Combined score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-04-07 19:15:44,416] A new study created in memory with name: Basic


Fold 1:
Epoch: 1, training loss: 1.1752
Epoch 1, Val Loss: 1.0746
Epoch: 2, training loss: 1.1142
Epoch 2, Val Loss: 1.0331
Epoch: 3, training loss: 1.0963
Epoch 3, Val Loss: 1.0771
Epoch: 4, training loss: 1.0960
Epoch 4, Val Loss: 1.0444
Epoch: 5, training loss: 1.0885
Epoch 5, Val Loss: 1.0640
Epoch: 6, training loss: 1.0821
Epoch 6, Val Loss: 1.0547
Epoch: 7, training loss: 1.0911
Epoch 7, Val Loss: 1.0512
Epoch: 8, training loss: 1.0913
Epoch 8, Val Loss: 1.0558
Epoch: 9, training loss: 1.0997
Epoch 9, Val Loss: 1.0401
Epoch: 10, training loss: 1.0930
Epoch 10, Val Loss: 1.0726
Fold 2:
Epoch: 1, training loss: 1.1849
Epoch 1, Val Loss: 1.1467
Epoch: 2, training loss: 1.1079
Epoch 2, Val Loss: 1.0722
Epoch: 3, training loss: 1.0763
Epoch 3, Val Loss: 1.0743
Epoch: 4, training loss: 1.0882
Epoch 4, Val Loss: 1.1518
Epoch: 5, training loss: 1.1009
Epoch 5, Val Loss: 1.0854
Epoch: 6, training loss: 1.0854
Epoch 6, Val Loss: 1.3694
Epoch: 7, training loss: 1.1273
Epoch 7, Val Loss: 1.0

[I 2025-04-07 19:15:50,856] Trial 0 finished with value: 0.24135359444644283 and parameters: {'hidden_dim': 102, 'dropout': 0.2865623578284173, 'initial_lr': 2.373445721330544e-05, 'max_lr': 0.09694282698403738, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 9}. Best is trial 0 with value: 0.24135359444644283.


Epoch: 9, training loss: 1.0676
Epoch 9, Val Loss: 1.1563
Epoch: 10, training loss: 1.0983
Epoch 10, Val Loss: 1.1275
Fold 1:
Epoch: 1, training loss: 1.2993
Epoch 1, Val Loss: 1.2350
Epoch: 2, training loss: 1.2369
Epoch 2, Val Loss: 1.1836
Epoch: 3, training loss: 1.2147
Epoch 3, Val Loss: 1.1433
Epoch: 4, training loss: 1.1701
Epoch 4, Val Loss: 1.1109
Epoch: 5, training loss: 1.1627
Epoch 5, Val Loss: 1.1072
Epoch: 6, training loss: 1.1452
Epoch 6, Val Loss: 1.1065
Epoch: 7, training loss: 1.1269
Epoch 7, Val Loss: 1.0970
Epoch: 8, training loss: 1.1314
Epoch 8, Val Loss: 1.1167
Epoch: 9, training loss: 1.1241
Epoch 9, Val Loss: 1.0971
Epoch: 10, training loss: 1.1209
Epoch 10, Val Loss: 1.1141
Fold 2:
Epoch: 1, training loss: 1.3143
Epoch 1, Val Loss: 1.2513
Epoch: 2, training loss: 1.2325
Epoch 2, Val Loss: 1.2010
Epoch: 3, training loss: 1.1929
Epoch 3, Val Loss: 1.1662
Epoch: 4, training loss: 1.1564
Epoch 4, Val Loss: 1.1411
Epoch: 5, training loss: 1.1392
Epoch 5, Val Loss: 1

[I 2025-04-07 19:15:55,996] Trial 1 finished with value: 0.223905539394924 and parameters: {'hidden_dim': 20, 'dropout': 0.23443747148705893, 'initial_lr': 0.0007176751647106228, 'max_lr': 0.03243199791432575, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 10}. Best is trial 0 with value: 0.24135359444644283.


...

Epoch: 10, training loss: 0.9211
Epoch 10, Val Loss: 0.9666
Fold 1:
Epoch: 1, training loss: 0.3583
Epoch 1, Val Loss: 0.3287
Epoch: 2, training loss: 0.3235
Epoch 2, Val Loss: 0.3139
Epoch: 3, training loss: 0.3070
Epoch 3, Val Loss: 0.3031
Epoch: 4, training loss: 0.2969
Epoch 4, Val Loss: 0.2853
Epoch: 5, training loss: 0.2911
Epoch 5, Val Loss: 0.2849
Epoch: 6, training loss: 0.2915
Epoch 6, Val Loss: 0.2827
Epoch: 7, training loss: 0.2913
Epoch 7, Val Loss: 0.2934
Epoch: 8, training loss: 0.2926
Epoch 8, Val Loss: 0.2868
Epoch: 9, training loss: 0.2899
Epoch 9, Val Loss: 0.2943
Epoch: 10, training loss: 0.2908
Epoch 10, Val Loss: 0.2894
Fold 2:
Epoch: 1, training loss: 0.3812
Epoch 1, Val Loss: 0.3282
Epoch: 2, training loss: 0.3211
Epoch 2, Val Loss: 0.3113
Epoch: 3, training loss: 0.3071
Epoch 3, Val Loss: 0.3009
Epoch: 4, training loss: 0.2971
Epoch 4, Val Loss: 0.2970
Epoch: 5, training loss: 0.2926
Epoch 5, Val Loss: 0.2960
Epoch: 6, training loss: 0.2932
Epoch 6, Val Loss: 0

[I 2025-04-07 19:16:22,971] Trial 6 finished with value: 0.3735721400524068 and parameters: {'hidden_dim': 77, 'dropout': 0.2975237045589857, 'initial_lr': 0.0008177914385283412, 'max_lr': 0.03359740840158348, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 1}. Best is trial 2 with value: 0.3762004050291111.


Epoch: 10, training loss: 0.2896
Epoch 10, Val Loss: 0.2962
Fold 1:
Epoch: 1, training loss: 1.3704
Epoch 1, Val Loss: 1.3306
Epoch: 2, training loss: 1.3505
Epoch 2, Val Loss: 1.3197
Epoch: 3, training loss: 1.3554
Epoch 3, Val Loss: 1.3092
Epoch: 4, training loss: 1.3425
Epoch 4, Val Loss: 1.2984
Epoch: 5, training loss: 1.3268
Epoch 5, Val Loss: 1.2892
Epoch: 6, training loss: 1.3134
Epoch 6, Val Loss: 1.2810
Epoch: 7, training loss: 1.3090
Epoch 7, Val Loss: 1.2736
Epoch: 8, training loss: 1.3052
Epoch 8, Val Loss: 1.2668
Epoch: 9, training loss: 1.2996
Epoch 9, Val Loss: 1.2596
Epoch: 10, training loss: 1.2820
Epoch 10, Val Loss: 1.2516
Fold 2:
Epoch: 1, training loss: 1.3588
Epoch 1, Val Loss: 1.3555
Epoch: 2, training loss: 1.3479
Epoch 2, Val Loss: 1.3518
Epoch: 3, training loss: 1.3487
Epoch 3, Val Loss: 1.3474
Epoch: 4, training loss: 1.3440
Epoch 4, Val Loss: 1.3420
Epoch: 5, training loss: 1.3363
Epoch 5, Val Loss: 1.3367
Epoch: 6, training loss: 1.3262
Epoch 6, Val Loss: 1

[W 2025-04-07 19:16:51,321] Trial 7 failed with parameters: {'hidden_dim': 18, 'dropout': 0.30357848033207613, 'initial_lr': 0.000422757556471017, 'max_lr': 0.00105278018378025, 'criterion': 'BCEWithLogitsLoss', 'pos_weight': 10} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "d:\GitHub repos\ADL\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_28972\3996966297.py", line 52, in maximise_combined_score
    model, accuracy, precision, recall, f1, auc = train_and_evaluate(
                                                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_28972\6689063.py", line 23, in train_and_evaluate
    for batch, (inputs, labels) in enumerate(train_loader,start=1):
  File "d:\GitHub repos\ADL\.venv\Lib\site-packages\torch\utils\data\dataloader.py", line 

KeyboardInterrupt: 